In [1]:
pip install prince

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install scikeras

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

from sklearn import *
import warnings
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from scipy import stats
from time import time as tt
np.random.seed(0)

In [6]:
from IPython.display import HTML
def pretty_print_df(value_counts_):
  "Quick function to display value counts more nicely"
  display(HTML(pd.DataFrame(value_counts_).to_html()))

In [9]:
data = pd.read_csv('../DataSet/OPTION1_uk_road_accident_2019_coursework_final.csv')

In [11]:
data.head()

accident_index  speed_limit light_conditions weather_conditions  \
0  2019010225080           30         darkness              other   
1  2019200908684           30         darkness               fine   
2  2019040860897           40         daylight               fine   
3  2019460847205           40         daylight               fine   
4  2019051911581           30         daylight               fine   

  road_surface_conditions      vehicle_type  \
0             wet or damp  at least one van   
1                     dry         only cars   
2                     dry         only cars   
3                     dry         only cars   
4                     dry         only cars   

                        junction_location    skidding_and_overturning  \
0      at or within 20 metres of junction  no skidding or overturning   
1      at or within 20 metres of junction  no skidding or overturning   
2      at or within 20 metres of junction  no skidding or overturning   
3  not at or within 20 metres of junction  no skidding or overturning   
4  not at or within 20 metres of junction  no skidding or overturning   

                vehicle_leaving_carriageway  \
0                  none leaving carriageway   
1  at least one vehicle leaving carriageway   
2                  none leaving carriageway   
3                  none leaving carriageway   
4                  none leaving carriageway   

           hit_object_off_carriageway  \
0                  none hit an object   
1  at least one vehicle hit an object   
2                  none hit an object   
3                  none hit an object   
4                  none hit an object   

                      first_point_of_impact                 sex_of_driver  \
0                    other points of impact                     all males   
1  at least one vehicle with frontal impact                     all males   
2  at least one vehicle with frontal impact  data missing or out of range   
3  at least one vehicle with frontal impact               male and female   
4  at least one vehicle with frontal impact                   all females   

   age_of_oldest_driver accident_severity  
0                  63.0           serious  
1                  82.0             fatal  
2                   NaN           serious  
3                   NaN           serious  
4                  39.0            slight

In [12]:
WorkingData = data.copy()

In [13]:
WorkingData.shape

(31647, 14)

In [14]:
display(WorkingData.columns)

Index(['accident_index', 'speed_limit', 'light_conditions',
       'weather_conditions', 'road_surface_conditions', 'vehicle_type',
       'junction_location', 'skidding_and_overturning',
       'vehicle_leaving_carriageway', 'hit_object_off_carriageway',
       'first_point_of_impact', 'sex_of_driver', 'age_of_oldest_driver',
       'accident_severity'],
      dtype='object')

In [16]:
WorkingData.describe()

speed_limit  age_of_oldest_driver
count  31647.000000          25197.000000
mean      36.572029             47.254038
std       13.837362             16.779656
min       -1.000000              6.000000
25%       30.000000             34.000000
50%       30.000000             47.000000
75%       40.000000             59.000000
max       70.000000            101.000000

In [17]:
WorkingData.dtypes

accident_index                  object
speed_limit                      int64
light_conditions                object
weather_conditions              object
road_surface_conditions         object
vehicle_type                    object
junction_location               object
skidding_and_overturning        object
vehicle_leaving_carriageway     object
hit_object_off_carriageway      object
first_point_of_impact           object
sex_of_driver                   object
age_of_oldest_driver           float64
accident_severity               object
dtype: object

In [18]:
WorkingData.isna().sum()

accident_index                    0
speed_limit                       0
light_conditions                  0
weather_conditions                0
road_surface_conditions           0
vehicle_type                      0
junction_location                 0
skidding_and_overturning          0
vehicle_leaving_carriageway       0
hit_object_off_carriageway        0
first_point_of_impact             0
sex_of_driver                     0
age_of_oldest_driver           6450
accident_severity              1172
dtype: int64

In [19]:
# let's list all categorical features
categorical_columns= ['light_conditions', 'weather_conditions', 'road_surface_conditions', 'vehicle_type','junction_location','skidding_and_overturning','vehicle_leaving_carriageway','hit_object_off_carriageway','first_point_of_impact',
       'accident_severity', 'sex_of_driver']
# Question: why have we not included Loan_ID?

# let's get the categories and their count for each feature
for col in categorical_columns:
  print(f"Categories and number of occurrences for '{col}'")
  pretty_print_df(WorkingData[col].value_counts())
  print()

Categories and number of occurrences for 'light_conditions'



Categories and number of occurrences for 'weather_conditions'



Categories and number of occurrences for 'road_surface_conditions'



Categories and number of occurrences for 'vehicle_type'



Categories and number of occurrences for 'junction_location'


...


Categories and number of occurrences for 'vehicle_leaving_carriageway'



Categories and number of occurrences for 'hit_object_off_carriageway'



Categories and number of occurrences for 'first_point_of_impact'



Categories and number of occurrences for 'accident_severity'



Categories and number of occurrences for 'sex_of_driver'


In [20]:
WorkingData.loc[WorkingData.accident_severity=='Slight','accident_severity'] = 'slight'
WorkingData.loc[WorkingData.accident_severity=='Serious','accident_severity'] = 'serious'
WorkingData.loc[WorkingData.accident_severity=='Fatal','accident_severity'] = 'fatal'

In [21]:
# replace out of scope value for the columns values with NaN
accepted_weather_conditions = ['fine','other','fog or mist']
accepted_road_surface_conditions = ['dry','wet or damp','other','flood over 3cm. deep']
accepted_vehicle_type = ['only cars','at least one biped','at least one van','biped and van','other']
accepted_junction_location = ['at or within 20 metres of junction','not at or within 20 metres of junction']
accepted_skidding_and_overturning = ['no skidding or overturning','at least one vehicle skidded or overturned']
accepted_vehicle_leaving_carriageway = ['none leaving carriageway','at least one vehicle leaving carriageway']
accepted_hit_object_off_carriageway = ['none hit an object','at least one vehicle hit an object']
accepted_first_point_of_impact = ['at least one vehicle with frontal impact','other points of impact','no impact']
accepted_sex_of_driver = ['all males','male and female','all females']


# remember the lambda notation: it is an anonymous function that lets us specificy
# a given (and easy) transformation for its input (in this case, x)
WorkingData.loc[WorkingData.weather_conditions.map(lambda x: x not in accepted_weather_conditions),'weather_conditions'] = np.nan
WorkingData.loc[WorkingData.road_surface_conditions.map(lambda x: x not in accepted_road_surface_conditions),'road_surface_conditions'] = np.nan
WorkingData.loc[WorkingData.vehicle_type.map(lambda x: x not in accepted_vehicle_type),'vehicle_type'] = np.nan
WorkingData.loc[WorkingData.junction_location.map(lambda x: x not in accepted_junction_location),'junction_location'] = np.nan
WorkingData.loc[WorkingData.skidding_and_overturning.map(lambda x: x not in accepted_skidding_and_overturning),'skidding_and_overturning'] = np.nan
WorkingData.loc[WorkingData.vehicle_leaving_carriageway.map(lambda x: x not in accepted_vehicle_leaving_carriageway),'vehicle_leaving_carriageway'] = np.nan
WorkingData.loc[WorkingData.hit_object_off_carriageway.map(lambda x: x not in accepted_hit_object_off_carriageway),'hit_object_off_carriageway'] = np.nan
WorkingData.loc[WorkingData.first_point_of_impact.map(lambda x: x not in accepted_first_point_of_impact),'first_point_of_impact'] = np.nan
WorkingData.loc[WorkingData.sex_of_driver.map(lambda x: x not in accepted_sex_of_driver),'sex_of_driver'] = np.nan
#replacing the -1 with an NaN
WorkingData.loc[WorkingData.speed_limit==-1,'speed_limit'] = np.nan

In [22]:
from sklearn.impute import SimpleImputer 

# data_no_nans =  data.copy()

categorical_columns= ['light_conditions', 'weather_conditions', 'road_surface_conditions', 'vehicle_type','junction_location','skidding_and_overturning','vehicle_leaving_carriageway','hit_object_off_carriageway','first_point_of_impact'
       , 'sex_of_driver']
### insert code here; work on the: train_dataset_no_nans
# 1. Imputer
imptr_cat = SimpleImputer(missing_values = np.nan, strategy = 'most_frequent')  

# 2. Fit the imputer object to the feature matrix (only for categorical features)
imptr_cat = imptr_cat.fit(WorkingData[categorical_columns])

# 3. Call Transform to replace missing data in train_dataset (on specific columns) by the mean of the column to which that missing data belongs to
WorkingData[categorical_columns] = imptr_cat.transform(WorkingData[categorical_columns])  


In [23]:
# 1. Imputer
imptr_num = SimpleImputer(missing_values = np.nan, strategy = 'median')  


# 2. Fit the imputer object to the feature matrix (only for numeric features)
numerical_columns = ['speed_limit', 'age_of_oldest_driver']
imptr_num = imptr_num.fit(WorkingData[numerical_columns]) # fit the data to estimate the parameters (here, the average value)

# 3. Call Transform to replace missing data in train_dataset (on specific columns) by the mean of the column to which that missing data belongs to
WorkingData[numerical_columns] = \
  imptr_num.transform(WorkingData[numerical_columns])